In [ ]:
import numpy as np
import random as rnd
import scipy.spatial.distance as distance
import math
import scipy.stats as stats
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plot
import scipy
import random
import itertools
import traceback
import matplotlib as mpl



In [ ]:
#Calculate mahalanobis distance
def myMahalanobis(x, mu, alpha):
    try:
        return abs(np.matmul((x - mu).transpose(),np.matmul(alpha, x - mu)))
    except ValueError:
        print("alpha = {}, x = {}, mu = {}".format(alpha, x, mu))

In [ ]:
# create a new component (representative of a multivariate gaussian distribution)
# TODO stop this form changing X
def create(accumulators, ages, alphas, comProbs, covDets, K, mus, sigmaInv, x):
    accumulators[K] = 1
    ages[K] = 1
    alphas[K] = sigmaInv * np.identity(len(alphas[0]))
    comProbs[K] = 1 / sum(accumulators.values())
    covDets[K] = np.linalg.det(alphas[K]) ** -1
    mus[K] = 0 + x
    #print(mus[K])
    x += 1
    ##print(mus[K])
    #print()
    x -= 1
    K += 1
    return K


In [ ]:
# update a component j using the posterior probabilities of data point x
#Algorithm 2 update
# TODO stop this from changing X
def update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, printing, x):
    #equation 22
    dist = (myMahalanobis(x, mus[j], alphas[j]))
    
    #equation 2
    pxj = 1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[j])) * math.exp(-0.5 * dist)#
   
    try:
        pxj = math.log(pxj)
        pxj = math.exp(pxj)
    except ValueError as err:
        print(str(err) + traceback.format_exc() + "dist = {}, pxj = {}, covDets[j] = {}".format(dist, pxj, covDets[j]))
    if printing:
        print(pxj)
    
    #equation 3
    pjx = pxj * comProbs[j]#
    totProbs = 0
    for i in range(K):
        pxi =  1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[i])) * math.exp(-0.5 * dist)#
        totProbs += pxi * comProbs[i]#
        
    if totProbs == 0:
        print(comProbs)
        print()
    try:
            pjx /= totProbs# 
    
    except ZeroDivisionError as err:
        print(str(err) + traceback.format_exc() + "pjx = {}, totProbs = {}".format(pjx, totProbs))
        
        
    if printing:
        print(pjx)
    
    #equation 4,5
    ages[j] += 1
    accumulators[j] += pjx
    
    #equations 6,7
    ej = x - mus[j]
    weight = pjx / accumulators[j]
    
    #equation 8,9
    deltaMu = weight * ej
    mus[j] += deltaMu

    oldAlpha = alphas[j]

    #equation 10
    ejStar = x - mus[j]
    
    #equation 20
    newAlpha = oldAlpha / (1 - weight) - (weight / (1 - weight) ** 2) * np.matmul(np.matmul(oldAlpha, ejStar), np.matmul(ejStar.transpose(), oldAlpha)) \
            / (1 + (weight / (1 - weight)) * np.matmul(np.matmul(ejStar.transpose(), oldAlpha),ejStar))
    
    #equation 21
    alphas[j] = newAlpha +  (np.matmul(np.matmul(newAlpha, mus[j]), np.matmul(mus[j].transpose(), newAlpha))) \
            / (1 - np.matmul(np.matmul(mus[j].transpose(), newAlpha), mus[j]))
        
    #equation 12
    comProbs[j] = accumulators[j] / sum(accumulators.values())
    
    #equations 25, 26
    newCovDet = (1 - weight) ** D * covDets[j] * (1 + weight/(1 - weight) * np.matmul(ejStar.transpose(), np.matmul(oldAlpha, ejStar)))
    covDets[j] = newCovDet * (1 - np.matmul(deltaMu.transpose(), np.matmul(newAlpha, deltaMu)))

    return mus
    

In [ ]:
# remove a component after it was merged with another component in the model
def remove(accumulators, ages, alphas, comProbs, correspondingData, covDets, j, mus, remainingK):
    correspondingData[-1].append(correspondingData[j])
    
    for i in range(j,len(ages) - 1):
        accumulators[i] = accumulators.pop(i + 1)
        ages[i] = ages.pop(i + 1)
        alphas[i] = alphas.pop(i + 1)
        comProbs[i] = comProbs.pop(i + 1)
        correspondingData[i] = correspondingData.pop(i + 1)
        covDets[i] = covDets.pop(i + 1)
        mus[i] = mus.pop(i + 1)
    
    comProbs = removeHelper(comProbs, accumulators)
    return comProbs

In [ ]:
#TODO find workaround for type to use logsumexp
def removeHelper(comProbs, accumulators):
    val = len(accumulators)
    for i in range(val):
        summed = 0
        for j in range(val):
            summed += accumulators[j]
        comProbs[i] = accumulators[i] / summed
    return comProbs

In [ ]:
# generate data from 5 normal gaussian distributions
#set plotting to True to see the scatter plots of generated data
#TODO plot sigmas of starting gaussians
def generateData(plotting = False):
    #TODO create dict to map key to data correspondence to mu/sigma, 4 keys, mu, sigma, discover mu, discover sigma 
    D = 4; K = 0; Beta = 0.05; delta = 0.01
    vmin = 5; spmin = 3
    mus = np.ndarray([D + 1,2]); sigmas = np.ndarray([D+1,2,2])
    mus = np.array([[1, 1], [10, 10], [10,1], [1,10], [5,5]])
    
    for i in range(D + 1):
        sigmas[i] = [[1, 0], [0, 1]]

    startingMus = mus
    numvectors = 250; lenvectors = 2
    X = np.ndarray((numvectors, lenvectors))



    for i in range(numvectors):
        index = i % (D + 1)
        
        for j in range(lenvectors):
            X[i] = np.random.multivariate_normal(mus[index], sigmas[index])
    if plotting:       
        Xs = [[],[],[],[],[]]
        Ys = [[],[],[],[],[]]
        colors = ['r','b','','y','m']

        for i in range(len(X)):
            index = i % 5
            Xs[index].append(X[i,0])
            Ys[index].append(X[i,1])

        for i in range(5):
            plot.scatter(Xs[i], Ys[i], c =colors[i])
            
    return [X, startingMus, sigmas]
# return the generated data, and the mus and sigmas of the gaussians used to generate t

In [ ]:
# generate data from 5 normal gaussian distributions
#set plotting to True to see the scatter plots of generated data
#TODO plot sigmas of starting gaussians
def generateData3(plotting = False):
    #TODO create dict to map key to data correspondence to mu/sigma, 4 keys, mu, sigma, discover mu, discover sigma 
    D = 4; K = 0; Beta = 0.05; delta = 0.01
    vmin = 5; spmin = 3
    mus = np.ndarray([D + 1,2]); sigmas = np.ndarray([D+1,2,2])
    mus = np.array([[1, 1], [10, 10], [10,1], [1,10], [5,5]])
    
    for i in range(D + 1):
        sigmas[i] = [[1, 0], [0, 1]]

    startingMus = mus
    numvectors = 500; lenvectors = 2
    X = np.ndarray((numvectors, lenvectors))
    startingCorrespondences = []


    for i in range(numvectors):
        index = random.randint(0,4)
        startingCorrespondences.append(index)
        X[i] = np.random.multivariate_normal(mus[index], sigmas[index])
        
    if plotting:       
        splot = plot.subplot(1, 1, 1)

        Xs = [[],[],[],[],[]]
        Ys = [[],[],[],[],[]]
        colors = ['r','b','k','y','m']

        for i in range(len(X)):
            index = startingCorrespondences[i]
            Xs[index].append(X[i,0])
            Ys[index].append(X[i,1])
        for i in range(5):
            plot.scatter(Xs[i], Ys[i], c =colors[i])
            v, w = np.linalg.eigh(sigmas[i])
            angle = np.arctan2(w[0][1], w[0][0])
            angle = 180.0 * angle / np.pi  # convert to degree
            Xs_to_plot = Xs[i]
            Ys_to_plot = Ys[i]
            plot.scatter(Xs_to_plot, Ys_to_plot, c = colors[i])
            plot.scatter((mus[i][0]), (mus[i][1]), c = 'g')

            ell = mpl.patches.Ellipse(mus[i],v[0], v[1], 180.0 + angle, color=colors[i % 5])
            ell.set_clip_box(splot.bbox)

            ell.set_alpha(0.5)
            splot.add_artist(ell)

    return [X, startingMus, sigmas, startingCorrespondences]
# return the generated data, and the mus and sigmas of the gaussians used to generate the data

In [ ]:
#TODO use Z score instead of mahalanobis distance to fall into 
# simulate the IGMM process for input data stream X
# set printing parameter to true to see final correspondences from the IGMM generated mus to the mus used in data generation
def simulate(X, sigmas, startingMus, delta = 1, dimension = 2, D = 4, vmin = 5, spmin = 3, printing = False):
    sigma = np.std(X[:10]) #sigma should be representative of only a part of the data stream
    sigmaInv = (delta * sigma) ** 2 ** -1
    alpha = sigmaInv * np.identity(dimension)
    covarDet = np.linalg.det(alpha) ** -1
    mDistMax = 20 #5 * sigma
    
    alphas = {}
    mus = {}
    covDets = {}
    comProbs = {}
    ages = {}
    accumulators ={}
    correspondingData = {}
    ZDict = {}

    alphas[0] = alpha
    covDets[0] = covarDet
    mus[0] = X[0]
    comProbs[0] = 1
    ages[0] = 1
    accumulators[0] = 1
    K = 1
    remainingK = D
    correspondingData[0] = [0]
    correspondingData[-1] = []
    correspondingData[-2] = []
    
    for i in range(len(X)):
        ZDict[i] = []
        if i == 0:
            continue
        updated = False

        for j in range(K):
            if j >= K:
                break
#             if (ages[j] > vmin) and (accumulators[j] < spmin):
#                 comProbs = remove(accumulators, ages, alphas, comProbs, correspondingData, covDets, j, mus, remainingK)
#                 remainingK += 1
#                 K -= 1
                
#                 if printing:
#                     print('call remove')
#                     print()

        for j in range(K):
            if i == j:
                continue

            
            dist = (myMahalanobis(X[i], mus[j], alphas[j]))
            if printing:
                print('(' + str(i) + ', ' + str(j) + ')')
                print(dist)
            if dist < mDistMax:
                if printing:
                    print('call update')
                    print()
                correspondingData[j].append(i)
                ZDict[i].append(getZScores(X[i], mus[j], np.linalg.inv(alphas[j]), dist))
                mus = update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, printing, X[i])
                #print('mu3 = {}'.format(mus[j]))
                #print()
                updated = True

            ##!!!!!! This may be problematic, only comparing means
        if K > 1:
            for i1 in range(K):
                for j1 in range(i):
                    if i1 == j1:
                        continue
                    if i1 < K and j1 < K:
                        mat = (np.linalg.inv(alphas[i1]) + np.linalg.inv(alphas[j1])) / 2
                        alp = np.linalg.inv(mat)
                        comDist = (myMahalanobis(mus[i1], mus[j1], alp))
                        if comDist < 0.2:
                            
                            #print('merged')
                            #print('i = {}, j = {}, meani = {}, meanj = {}'.format(i1, j1, mus[i1], mus[j1]))
                            #print()
                            for k in correspondingData[j1]:
                                #print('correspondingData = {}'.format(correspondingData[i1]))
                                #print('correspondingData = {}'.format(correspondingData[j1]))
                                if k not in correspondingData[i1]:
                                    dist = (myMahalanobis(X[k], mus[i1], alphas[i1]))
                                    if dist < mDistMax:
                                        ZDict[k].append(getZScores(X[k], mus[i1], np.linalg.inv(alphas[i1]), dist))
                                        mus = update(accumulators, ages, alphas, comProbs, covDets, D, i1, K, mus, printing, X[k])
                                        correspondingData[i1].append(k)
                                    else:
                                        correspondingData[-2].append(k)
                                #print('correspondingData = {}\n'.format(correspondingData[i1]))
                            comProbs = remove(accumulators, ages, alphas, comProbs, correspondingData, covDets, j1, mus, remainingK)
                            remainingK += 1
                            K -= 1
         ## end !!!!!!
    
        
        if not updated and remainingK == 0:
            correspondingData[-1].append(i)
        
        if not updated and not remainingK == 0:
            if printing:
                print('call create')
                print()
            correspondingData[K] = [i]
            K = create(accumulators, ages, alphas, comProbs, covDets, K, mus, sigmaInv, X[i])
            remainingK -= 1
            

            
    [alphas, correspondingData, mus, K, comProbs, covDets, ZDict, oldZDict] = simulateMature(accumulators, ages, alphas, comProbs, correspondingData, covDets, D, K, mus, X[correspondingData[-1]], ZDict, mDistMax = 20)

    if printing:
        print(K)
    norms = np.ndarray([D + 1, K])
    difference = 0
    correspondences = np.ndarray(D + 1)
    for i in range(D + 1): #D + 1 = len(startingMus)
        for j in range(K):
            norms[i, j] = np.linalg.norm(startingMus[i] - mus[j])
        index = np.argmin(norms[i])
        difference += norms[i,index]
        correspondences[i] = index
        
        if printing:
            print("mu = {}, sigma = {}".format(mus[index], np.linalg.inv(alphas[index])))
            print("starting mu = {}, starting Sigma = {}".format(startingMus[i], sigmas[i]))
            print("corresponding data = {}\n".format(correspondingData[index]))


        #TODO handle case when
    if printing:
        print(str(difference) + '\n')
        print(correspondingData)
            

    return [alphas, correspondingData, difference, mus, K, comProbs, covDets, ZDict, oldZDict]


In [ ]:
# simulate the IGMM process updates for all outliers
# set printing parameter to true to see final correspondences from the IGMM generated mus to the mus used in data generation
def simulateMature(accumulators, ages, alphas, comProbs, correspondingData, covDets, D, K, mus, outliers, ZDict, mDistMax = 11, printing = False):
    X = outliers
    oldZDict = ZDict
    ZDict = {}
    correspondingData[-3] = []
    for i in range(len(X)):
        ZDict[i] = []
        updated = False
        distArr = []
        for j in range(K):
            dist = (myMahalanobis(X[i], mus[j], alphas[j]))
            distArr.append(dist)
            if printing:
                print('(' + str(i) + ', ' + str(j) + ')')
                print(dist)
            if dist < mDistMax:
                if printing:
                    print('call update')
                    print()
                correspondingData[j].append(i)
                ZDict[i].append(getZScores(X[i], mus[j], np.linalg.inv(alphas[j]), dist))
                mus = update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, printing, X[i])
                updated = True

            ##!!!!!! This may be problematic, only comparing means
#         if K > 1:
#             for i1 in range(K):
#                 for j1 in range(i):
#                     if i1 == j1:
#                         continue
#                     if i1 < K and j1 < K:
#                         mat = (np.linalg.inv(alphas[i1]) + np.linalg.inv(alphas[j1])) / 2
#                         alp = np.linalg.inv(mat)
#                         comDist = (myMahalanobis(mus[i1], mus[j1], alp))
#                         if comDist < 0.2:
                            
#                             #print('merged')
#                             #print('i = {}, j = {}, meani = {}, meanj = {}'.format(i1, j1, mus[i1], mus[j1]))
#                             #print()
#                             for k in correspondingData[j1]:
#                                 #print('correspondingData = {}'.format(correspondingData[i1]))
#                                 #print('correspondingData = {}'.format(correspondingData[j1]))
#                                 if k not in correspondingData[i1]:
#                                     dist = (myMahalanobis(X[k], mus[i1], alphas[i1]))
#                                     if dist < mDistMax:
#                                         ZDict[k].append(getZScores(X[k], mus[i1], np.linalg.inv(alphas[i1]), dist))
#                                         mus = update(accumulators, ages, alphas, comProbs, covDets, D, i1, K, mus, printing, sigmaInv, X[k])
#                                         correspondingData[i1].append(k)
#                                     else:
#                                         correspondingData[-2].append(k)
#                                 #print('correspondingData = {}\n'.format(correspondingData[i1]))
#                             comProbs = remove(accumulators, ages, alphas, comProbs, correspondingData, covDets, j1, mus, remainingK)
#                             remainingK += 1
#                             K -= 1
         ## end !!!!!!
    
        
        if not updated and min(distArr) < 11:
            
            correspondingData[j].append(i)
            ZDict[i].append(getZScores(X[i], mus[j], np.linalg.inv(alphas[j]), dist))
            mus = update(accumulators, ages, alphas, comProbs, covDets, D, j, K, mus, printing, X[i])
        elif not updated:
            correspondingData[-3].append(i)
    
    return [alphas, correspondingData, mus, K, comProbs, covDets, ZDict, oldZDict]

In [ ]:
# Takes in an array of outliers and calculates their posterior probabilities for each of the gaussians produced by the IGMM

def getSoftProbsOut(alphas, comProbs, covDets, K, mus, outliers, Xini):
    X = np.copy(Xini)
    D = K - 1
    softProbs = np.ndarray([len(outliers),K])
    probs = np.ndarray([K])
    for i in range(len(outliers)):
        for j in range(K):
            dist = (myMahalanobis(X[i], mus[j], alphas[j]))
            probs[j] = 1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[j])) * math.exp(-0.5 * dist) * comProbs[j]

        probsum = sum(probs)
        
        for j in range(K):
            softProbs[i,j] = probs[j] / probsum
    return softProbs
        
    

In [ ]:
# simulate the igmm and calculate the distance from the resulting means to the original means, then roll the input matrix
# and repeat until we minimize the distance between the resulting means and original means
# number of trials determines how many times we roll the matrix and test
# TODO verify this output matches expected differences and correctly finds optimal starting data point
# TODO fine tune this to ensure proper input is used during each trial and is not altered during trials
def sieve(Data, K = 5, D = 4, printing = False, trials = 50): 
    
    savedData = np.copy(Data)
    differences = np.ndarray(trials)
    X = Data[0]; sigmas = Data[2]; startingMus = Data[1]
    
    for t in range(trials):
        [alphas, correspondingData, difference, mus, K, comProbs, covDets, zDict] = simulate(X = Data[0], sigmas = Data[2], startingMus = Data[1], printing = False)

    
        norms = np.ndarray([D + 1, K])
        difference = 0
        correspondences = np.ndarray(D + 1)

        for i in range(D + 1): #D + 1 = len(startingMus)
            for j in range(K):
                norms[i, j] = (np.linalg.norm(startingMus[i] - mus[j])) ** 2
            index = np.argmin(norms[i])
            difference += norms[i,index]
            correspondences[i] = index
            if printing:    
                print("mu = {}, sigma = {}".format(mus[index], np.linalg.inv(alphas[index])))
                print("starting mu = {}, starting Sigma = {}".format(startingMus[i], sigmas[i]))
                print("corresponding data = {}\n".format(correspondingData[index]))


        if printing:
            print(str(difference) + '\n')
            print(correspondingData)

        if t == 0:
            mindifference = difference
            besti = 0

        elif mindifference > difference:
            mindifference = difference
            besti = t

        differences[t] = difference

        Data = np.copy(savedData)
        X = np.roll(Data[0],1); sigmas = Data[2]; startingMus = Data[1]


    return [mindifference, besti, differences]

In [ ]:
def getSoftProbs(alphas, comProbs, covDets, K, mus, Xini):
    X = np.copy(Xini)
    D = K - 1
    softProbs = np.ndarray([len(Xini),K])
    probs = np.ndarray([K])
    for i in range(len(Xini)):
        for j in range(K):
            dist = (myMahalanobis(X[i], mus[j], alphas[j]))
            probs[j] = 1 / ((2 * math.pi) ** (D / 2) * math.sqrt(covDets[j])) * math.exp(-0.5 * dist) * comProbs[j]

        probsum = sum(probs)
        
        for j in range(K):
            try:
                softProbs[i,j] = math.exp (math.log(probs[j]) - math.log(probsum))
            except ValueError:
                softProbs[i,j] = 0
    return softProbs

In [ ]:
def getZScores(x, mu, sigma, dist):
    ZScores = [dist]
    for i in range(len(x)):
        ZScores.append(((x[i] - mu[i]) / sigma[i,i]))
    return ZScores

In [ ]:
Data = generateData3(True) #generate some data without plotting it
Xini = np.copy(Data[0]) #save a copy of the input datastream (avoid losing it during update)

# sievedData = sieve(Data, printing = False) # sieve the data to determine the ideal starting input
# Xini = np.roll(Xini, sievedData[1])
# Data[3] = np.roll(Data[3],sievedData[1])
# Data[0] = np.copy(Xini)
[alphas, correspondingData, difference, mus, K, comProbs, covDets, ZDict, oldZDict] = simulate(X = Data[0], sigmas = Data[2], startingMus = Data[1], printing = False)


newSigmas = []
for i in range(len(alphas)):
    newSigmas.append(np.linalg.inv(alphas[i]))
print(newSigmas)
M = GaussianMixture(n_components = 5, covariance_type = 'full').fit(np.copy(Xini))
print(M.means_)
print(mus)
print()

#softProbs = getSoftProbs(alphas, comProbs, covDets, K, mus, Xini[:100])
#print(len(softProbs))
#print(softProbs)
#print()

#print(M.predict(Xini[:100]))
#print the results
#print(sievedData[0])
#print(sievedData[1])
#print(min(sievedData[2]))
#print(sievedData[2])
#print(0 in sievedData[2])

#[alphas, correspondingData, difference, mus, K, comProbs, covDets] = simulate(X = Data[0], sigmas = Data[2], startingMus = Data[1], printing = False)
#alphas = simulatedData[0]
#correspondingData = simulatedData[1]
#mus = simulatedData[3]



In [ ]:
#Some code for plotting that needs to be turned into a new funtion
colors = ['r', 'b', 'y', 'k', 'm']
for i in range(len(mus)):
    plot.figure(1)
    A = correspondingData[i]
    
    Xs_to_plot = Xini[A,0]
    Ys_to_plot = Xini[A,1]
    plot.scatter(Xs_to_plot, Ys_to_plot, c = colors[i % 5])
    plot.scatter((mus[i][0]), (mus[i][1]), c = 'g')


In [ ]:
for i in range(5):
    print(len(correspondingData[i]))
    


In [ ]:
print(correspondingData[-3])

In [ ]:
# splot = plot.subplot(2, 1, 2)

# color_iter = itertools.cycle(["navy", "turquoise", "cornflowerblue", "darkorange"])
# for i, (mean, cov, color) in enumerate(zip(mus, newSigmas, color_iter)):
#     v, w = np.linalg.eigh(cov)
# #     if not np.any(Y_ == i):
# #         continue
# #    plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], 0.8, color=color)

#     # Plot an ellipse to show the Gaussian component
#     angle = np.arctan2(w[0][1], w[0][0])
#     angle = 180.0 * angle / np.pi  # convert to degrees
#     v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
#     ell = mpl.patches.Ellipse(mean, v[0], v[1], 180.0 + angle, color=color)
#     ell.set_clip_box(splot.bbox)
#     ell.set_alpha(0.5)
#     splot.add_artist(ell)

In [ ]:
matchedMus = {}
newAlphas = {}
newComProbs = {}
newCovDets = {}

for i in range(len(mus)):
    distances = []
    for j in range(len(M.means_)):
        distances.append(np.linalg.norm(M.means_[j] - mus[i]))
    matchedMus[distances.index(min(distances))] = mus[i]
    newAlphas[distances.index(min(distances))] = alphas[i]
    newComProbs[distances.index(min(distances))] = comProbs[i]
    newCovDets[distances.index(min(distances))] = covDets[i]

newSigmas = {}
for i in range(len(newAlphas)):
    newSigmas[i] = np.linalg.inv(newAlphas[i])


newData = generateData3(False)#generate some data without plotting i
Xnew = newData[0]
softProbs = getSoftProbs(newAlphas, newComProbs, newCovDets, K, matchedMus, Xnew)
#print(softProbs)
sPArr = []
for i in range(len(softProbs)):
    sPArr.append(np.argmax(softProbs[i]))

#print(sPArr)
batch = M.predict(Xnew)
#print(batch)

startingMus = newData[1]
startingSigmas = newData[2]
startingCorrespondences = newData[3]

matchedStartingSigmas = {}
matchedStartingMus = {}
componentMap = {}
for i in range(len(startingMus)):
    distances = []
    for j in range(len(M.means_)):
        distances.append(np.linalg.norm(startingMus[i] - M.means_[j]))
    matchedStartingMus[distances.index(min(distances))] = startingMus[i]
    matchedStartingSigmas[distances.index(min(distances))] = startingSigmas[i]
    componentMap[i] = (distances.index(min(distances)))
startingMus = matchedStartingMus
startingSigmas = matchedStartingSigmas
    
for i in range(len(mus)):
    print("{}th starting mu = {}".format(i,startingMus[i]))
    print("{}th incremental mu = {}".format(i,matchedMus[i]))
    print("{}th batch mu = {}".format(i, M.means_[i]))
    print()
    print("{}th starting sigma = {}".format(i, startingSigmas[i]))
    print("{}th incremental sigma = {}".format(i, newSigmas[i]))
    print("{}th batch sigma = {}".format(i, M.covariances_[i]))
    print("\n\n")

for i in range(len(startingCorrespondences)):
    startingCorrespondences[i] = componentMap[startingCorrespondences[i]]

#print(startingCorrespondences)
bCount = 0
iCount = 0
for i in range(len(sPArr)):
    if sPArr[i] == startingCorrespondences[i]:
        iCount += 1
    if batch[i] == startingCorrespondences[i]:
        bCount += 1
iAcc = iCount / len(startingCorrespondences) * 100
bAcc = bCount / len(startingCorrespondences) * 100
print("The IGMM was {}% accurate.".format(iAcc))
print("The batch GMM was {}% accurate.".format(bAcc))


In [ ]:
print(startingMus)
print(startingSigmas)

In [ ]:
print(newSigmas)

In [ ]:
count = 0
for i in softProbs:
    if (i == np.zeros(5)).all():
        count += 1
print(count)
print(len(softProbs))

In [ ]:
i = softProbs[0]
print(i == [0.,0.,0.,0.,0.])
print(i == np.zeros(5))

In [ ]:
print(Xini[5])
print()
for z in ZDict:
    if ZDict[z] != []:
        print("Zdict[{}] = {}".format(Xini[z], ZDict[z]))


In [ ]:
dictionary = {1:2, 2:3, 3:4}
oldDict = dictionary
dictionary = {}
print(oldDict)

In [ ]:
arr = []
for i in range(len(mus)):
    arr.append(mus[i])
print(mus)
matchedMus = {}
for i in range(len(mus)):
    distances = []
    for j in range(len(M.means_)):
        distances.append(np.linalg.norm(M.means_[j] - arr[i]))
    matchedMus[distances.index(min(distances))] = arr[i]
    
print(matchedMus)
print(M.means_)